In [1]:
import sys
import os

# Import utils from subfolder of project, works for immediate subfolders of PROJECT_ROOT
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..")) # adjust relative import as necessary
sys.path.append(PROJECT_ROOT)
from utils.data_processing import get_filtered_review_data, get_metadata

In [2]:
categorical = ['store', 'categories']
continuous = ['average_rating', 'rating_number', 'price']
text_based = ['title', 'features', 'description']

SELECTED_CATEGORY = 'Video_Games'
X_train, y_train, X_val, y_val, X_test, y_test = get_filtered_review_data(SELECTED_CATEGORY)
meta = get_metadata(SELECTED_CATEGORY)

Loading preprocessed data from data/Video_Games_min5_test1_val1_cols['user_id', 'product_id', 'timestamp', 'title', 'text', 'helpful_vote'].pkl
Loading metadata from data/Video_Games_metadata.pkl


In [3]:
meta.head()

,main_category,title,average_rating,rating_number,features,description,price,store,categories,details,parent_asin
0,Video Games,Dash 8-300 Professional Add-On,1.000,0.000000,Features Dash 8-300 and 8-Q300 ('Q' rollout li...,The Dash 8-300 Professional Add-On lets you pi...,NaN,Aerosoft,"[Video Games, PC, Games]",Pricing: The strikethrough price is the List P...,B000FH0MHO
1,Video Games,Phantasmagoria: A Puzzle of Flesh,0.775,0.000061,Windows 95,,NaN,Sierra,"[Video Games, PC, Games]","Best Sellers Rank: Video Games: 137612, PC-com...",B00069EVOG
2,Video Games,NBA 2K17 - Early Tip Off Edition - PlayStation 4,0.825,0.000797,The #1 rated NBA video game simulation series ...,Following the record-breaking launch of NBA 2K...,0.016571,2K,"[Video Games, PlayStation 4, Games]","Release date: September 16, 2016, Best Sellers...",B00Z9TLVK0
3,Video Games,Nintendo Selects: The Legend of Zelda Ocarina ...,0.975,0.000075,Authentic Nintendo Selects: The Legend of Zeld...,,0.010691,Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems...","Best Sellers Rank: Video Games: 51019, Nintend...",B07SZJZV88
4,Video Games,Thrustmaster Elite Fitness Pack for Nintendo Wii,0.500,0.000007,Includes (9) Total Accessories Pedometer Wii F...,The Thrustmaster Motion Plus Elite Fitness Pac...,NaN,THRUSTMASTER,"[Video Games, Legacy Systems, Nintendo Systems...","Release date: November 1, 2009, Pricing: The s...",B002WH4ZJG


In [ ]:
from openai import OpenAI
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("API_KEY")
ORGANIZATION = os.getenv("ORGANIZATION")
PROJECT = os.getenv("PROJECT")

client = OpenAI(
  organization=ORGANIZATION,
  project=PROJECT,
  api_key=API_KEY
)

response = client.embeddings.create(
    model="text-embedding-3-small", input="example_string", encoding_format="float", 
)

cut_dim = response.data[0].embedding[:256]
print(cut_dim)

In [ ]:
def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)

norm_dim = normalize_l2(cut_dim)
print(norm_dim)

In [4]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("text-embedding-3-small")

In [4]:
from sentence_transformers import SentenceTransformer

# Load a pretrained model optimized for similarity
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight & fast

# Example product descriptions
product_descriptions = [
    "Wireless Bluetooth headphones with noise cancellation.",
    "Over-ear headphones with deep bass and long battery life.",
    "Organic green tea with a refreshing taste."
]

# Generate vector embeddings
embeddings = model.encode(product_descriptions, convert_to_tensor=True)

# Print the shape of the embeddings (should be [num_products, embedding_dim])
print(embeddings.shape)  # Example: torch.Size([3, 384])


torch.Size([3, 384])
